<a href="https://colab.research.google.com/github/invegat/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



In [1]:
!pip install geopy

## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [2]:
!pip install geopy

In [3]:

!pip install beautifulsoup4

In [0]:

from requests import get
from contextlib import closing
from bs4 import BeautifulSoup
import bs4
import re
import regex
import pandas as pd
from geopy.distance import distance
import datetime

In [0]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [0]:

#url = "https://ftw.usatoday.com/2016/08/ranking-best-worst-mlb-stadiums-30-petco-wrigley-pnc"
url = "https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums"
raw_html = simple_get(url)
html = BeautifulSoup(raw_html, 'html.parser')

table = html.select('table')[1]
teams = []
stadium = None
for tr in table.select('tr'):
    if tr.has_attr('style'):
        # print('tr style', tr['style'])
        tds = [td for td in tr.select('td')]
        if len(tds) > 6:
            teams.append((tds[5].a.text, tds[1].a.text))
    else :
        stadium = tr.find_next('th', {'scope': 'row'}).a.text
        tds = [td for td in tr.select('td')]
        if len(tds) > 5:
            team = tds[4].a.text
            teams.append((team, stadium))
            if team == 'New York Yankees':
                break 


In [12]:

gurl = "https://en.wikipedia.org/wiki/"
p = regex.compile(r"(\-?[\d\.]+)\;\s+(\-?[\d\.]+)")
lat = 0
long = 0
rows = []
# teams = [('Arizona Diamondbacks','Chase Field' )]
# teams = sorted(teams, key = lambda team: team[1])
for team, stadium in teams:
    # print(team, stadium)
    raw_html = simple_get(gurl  + stadium.replace(' ', '_')).decode("utf-8") 
    html = BeautifulSoup(raw_html, 'html.parser')
    geo = html.select('span .geo')
    # print(f"geo: {geo}")
    m = regex.match(p, geo[0].text)
    print(geo[0].text)
    rows.append({'team': team, 'stadium': stadium, 'lat': float(m.group(1)), 'long': float(m.group(2))})
teamd = pd.DataFrame(rows)   
teamd

33.80028; -117.88278
38.62250; -90.19306
33.44528; -112.06694
40.75694; -73.84583
39.90583; -75.16639
42.33917; -83.04861
39.75611; -104.99417
34.07361; -118.24000
42.346250; -71.097750
32.75139; -97.08278
39.09750; -84.50667
41.83000; -87.63389
39.05139; -94.48056
25.77806; -80.21972
43.02833; -87.97111
29.75694; -95.35556
38.87278; -77.00750
37.75167; -122.20056
37.77861; -122.38917
39.28389; -76.62167
32.7073; -117.1566
40.44694; -80.00583
41.49583; -81.68528
43.64139; -79.38917
33.89; -84.468
47.591; -122.333
44.98167; -93.27833
27.76833; -82.65333
41.948038; -87.65568
40.82917; -73.92639


,lat,long,stadium,team
0,33.800280,-117.88278,Angel Stadium,Los Angeles Angels
1,38.622500,-90.19306,Busch Stadium,St. Louis Cardinals
2,33.445280,-112.06694,Chase Field,Arizona Diamondbacks
3,40.756940,-73.84583,Citi Field,New York Mets
4,39.905830,-75.16639,Citizens Bank Park,Philadelphia Phillies
5,42.339170,-83.04861,Comerica Park,Detroit Tigers
6,39.756110,-104.99417,Coors Field,Colorado Rockies
7,34.073610,-118.24000,Dodger Stadium,Los Angeles Dodgers
8,42.346250,-71.09775,Fenway Park,Boston Red Sox
9,32.751390,-97.08278,Globe Life Park in Arlington,Texas Rangers


In [0]:

def print_distance(winner, loser, rows):
    w = teamd[teamd['team'] == winner]
    l = teamd[teamd['team'] == loser]
    if len(w.lat.values) == 0:
        print('unknown winning team', winner)
    if len(l.lat.values) == 0:
        print('unknown losing team', loser)                  
    d = distance((w.lat.values[0], w.long.values[0]),
                 (l.lat.values[0], l.long.values[0])).miles
    rows.append({'year': year, 'winner': winner, 'loser': loser, 'distance': d})

In [8]:
wsurl = "https://en.wikipedia.org/wiki/List_of_World_Series_champions"
raw_html = simple_get(wsurl)
html = BeautifulSoup(raw_html, 'html.parser')
win = True
winner = None
rows = []
year = datetime.datetime.now().year
for tr in html.select('tr'):
    th = tr.find_next('th', attrs={'scope':'row'})
    a = th.a
    series_year = int(a.text)
#     if series_year == year:
         # necessary iff world series hasn't been decided yet this year (picked my team to win)
         # edit in the teams
#         print_distance('Boston Red Sox', 'Los Angeles Dodgers', rows)
#         break
    if series_year > 1958: # before 1958 no west coast teams so no maximum distance       
        for td in tr.select('td'):
            if (td.has_attr('style')):
                tda = td.a
                if win:
                    winner = tda.text
                    if winner == 'Florida Marlins':  # convert old name to current name
                        winner = 'Miami Marlins'
                    elif winner == 'Anaheim Angels': # convert old name to current name
                        winner = 'Los Angeles Angels'
                else:
                    print_distance(winner, tda.text, rows)
                win = False if win else True
    if series_year == year:
        # necessary iff world series has been decided this year
        break
# print(len(rows))
distances = pd.DataFrame(rows)
# distances

ValueError: ignored

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things
print('Mark Oliver')
1+1

Mark Oliver


2

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about